In [1]:
import requests
from bs4 import BeautifulSoup
import random
from tqdm import tqdm
import pandas


Should you go to Mines or Boulder?

In [8]:
api_url = 'https://api.data.gov/ed/collegescorecard/v1/schools'
api_key='oqaiUZhYmvyPfyIw43pYx3Smga15vRSWVhLDIJlg'

career_length=35
college_length=7
school_names = ["Colorado School of Mines", "University of Colorado Boulder"]

data = {
    'College': school_names,
    'Median Debt (CS)': [],
    'Median Earnings (CS)': [],
}


def get_college_data(school_name):
    params = {
        'api_key': api_key,
        'school.name': school_name,
        'fields': 'id,school.name,latest.median_debt,latest.median_earnings,fields_of_study'
    }
    response = requests.get(api_url, params=params)

    # Print the HTTP response status code and full response for debugging
    print(f"HTTP Response Status Code: {response.status_code}")
    print(f"Full API Response: {response.text}")  # Print the raw response text

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error fetching data for {school_name}: {response.status_code}")
        return None

# Iterate over each school
for i, school_name in enumerate(data['College']):
    # Step 1: Fetch college data from the API
    college_data = get_college_data(school_name)
    if college_data:
        # Extract the relevant data from the API response
        median_debt = college_data.get('latest', {}).get('median_debt', 'Data not available')
        median_earnings = college_data.get('latest', {}).get('median_earnings', 'Data not available')

        # Filter for Computer Science or related fields in 'fields_of_study'
        if 'fields_of_study' in college_data:
            found_cs = False
            for program in college_data['fields_of_study']:
                if 'Computer Science' in program.get('program_name', '').lower():
                    median_debt = program.get('median_debt', 'Data not available')
                    median_earnings = program.get('median_earnings', 'Data not available')
                    found_cs = True
                    break
            
            # If no Computer Science field found, keep the general data
            if not found_cs:
                median_debt = 'Computer Science data not available'
                median_earnings = 'Computer Science data not available'

        data['Median Debt (CS)'].append(median_debt)
        data['Median Earnings (CS)'].append(median_earnings)

        print(f"API Data for {school_name}:")
        print(f"  Median Debt (CS): {median_debt}")
        print(f"  Median Earnings (CS): {median_earnings}")
    else:
        data['Median Debt (CS)'].append('Data not available')
        data['Median Earnings (CS)'].append('Data not available')

# Print collected data
print("\nCollected Data:")
print(data)

HTTP Response Status Code: 200
Full API Response: {"metadata":{"page":0,"total":1,"per_page":20},"results":[{"school.name":"Colorado School of Mines","id":126775}]}
API Data for Colorado School of Mines:
  Median Debt (CS): Data not available
  Median Earnings (CS): Data not available
HTTP Response Status Code: 200
Full API Response: {"metadata":{"page":0,"total":1,"per_page":20},"results":[{"school.name":"University of Colorado Boulder","id":126614}]}
API Data for University of Colorado Boulder:
  Median Debt (CS): Data not available
  Median Earnings (CS): Data not available

Collected Data:
{'College': ['Colorado School of Mines', 'University of Colorado Boulder'], 'Median Debt (CS)': ['Data not available', 'Data not available'], 'Median Earnings (CS)': ['Data not available', 'Data not available']}
